In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ayuraj/asl-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\remir\.cache\kagglehub\datasets\ayuraj\asl-dataset\versions\1


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lexset/synthetic-asl-alphabet")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\remir\.cache\kagglehub\datasets\lexset\synthetic-asl-alphabet\versions\3


In [3]:
# Download latest version
path = kagglehub.dataset_download("signnteam/asl-sign-language-pictures-minus-j-z")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\remir\.cache\kagglehub\datasets\signnteam\asl-sign-language-pictures-minus-j-z\versions\1


In [4]:
#data = "C:/Users/remir/.cache/kagglehub/datasets/ayuraj/asl-dataset/versions/1/asl_dataset"
#data = "C:/Users/remir/.cache/kagglehub/datasets/lexset/synthetic-asl-alphabet/versions/3/Train_Alphabet"
data = "C:/Users/remir/.cache/kagglehub/datasets/signnteam/asl-sign-language-pictures-minus-j-z/versions/1/asl_dataset"

import os
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd

In [5]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)

def extract_landmarks(image_path):
    image = cv2.imread(image_path)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_image)
    if results.multi_hand_landmarks:
        landmarks = results.multi_hand_landmarks[0]
        return [(lm.x, lm.y, lm.z) for lm in landmarks.landmark]
    return None  

dataset_dir = data
output_data = []
labels = []

for class_dir in os.listdir(dataset_dir):
    class_path = os.path.join(dataset_dir, class_dir)
    if os.path.isdir(class_path):
        for image_file in os.listdir(class_path):
            image_path = os.path.join(class_path, image_file)
            landmarks = extract_landmarks(image_path)
            if landmarks is not None:
                output_data.append(np.array(landmarks).flatten())
                labels.append(class_dir)

output_df = pd.DataFrame(output_data)
output_df['label'] = labels
output_df.to_csv("landmarks_dataset.csv", index=False)

In [6]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = pd.read_csv("landmarks_dataset.csv")
X = data.iloc[:, :-1]  
y = data['label']     

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.9398677089597114


In [7]:
import joblib
joblib.dump(rf,'randomForest.joblib')

['randomForest.joblib']

In [8]:
import cv2
import mediapipe as mp
import numpy as np

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)

roi_width = 175
roi_height = 175
roi_x = 40  
roi_y = 20  

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Impossible de lire la caméra")
        break

    # Region of interest
    start_point = (roi_x, roi_y)
    end_point = (roi_x + roi_width, roi_y + roi_height)
    color = (255, 0, 0)  
    thickness = 2 
    cv2.rectangle(frame, start_point, end_point, color, thickness)

    roi_frame = frame[roi_y:roi_y + roi_height, roi_x:roi_x + roi_width]
    rgb_roi_frame = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_roi_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks = [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]
            features = np.array(landmarks).flatten()

            prediction = rf.predict([features])[0]

            cv2.putText(frame, f"Signe : {prediction}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow('Detection de signes', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

c:\Users\remir\Documents\CPE\5ETI\Majeure\env_msi\env_msi\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\remir\Documents\CPE\5ETI\Majeure\env_msi\env_msi\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\remir\Documents\CPE\5ETI\Majeure\env_msi\env_msi\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\remir\Documents\CPE\5ETI\Majeure\env_msi\env_msi\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\remir\Documents\CPE\5ETI\Majeure\env_msi\env_msi\Lib\site-packages\sklearn\base.py:493: UserWarning: X does